# Analyzing Data with IBM Db2, SQL, and Python
## George Jordan
*Last update: January 13, 2021*


I am currently working through the 'IBM Data Science Professional Certificate' through Coursera. The certificate is a nine-course suite, with the fifth course being 'Databases and SQL for Data Science'. I have recently completed this course and will use the work here to demonstrate my skills in these tools.

First, I will provide a brief overview of the queries that I wrote to fulfill the requirements of the assignment (which scored 100%). Then, I will visually display the data by comparing variables via scatterplot. I will use regression analysis to describe the relationships between variables. In doing so, I hope to provide interpretation that might otherwise not be obvious by simply viewing the data within the databases.

I believe that data does not speak for itself and requires a storyteller to communicate findings so that stakeholders can make informed decisions with the data. I firmly believe that this work is a waste of time if teams cannot use the work either for lack of interpretation or techincal knowledge. I hope that I can be that bridge to connect the data to those otherwise unable to use it.

In this project, I used three separate data sets to gain insight into Chicago's neighborhoods and schools. The data is made available through the city of Chicago's public data portal. For the scope of the project, the crime data has been truncated to around five hundred rows, as opposed to the original which contained around 6.5 million rows.

1.  <a href="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2">Socioeconomic Indicators in Chicago</a>
2.  <a href="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t">Chicago Public Schools</a>
3.  <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2">Chicago Crime Data</a>

### Preparation

In [1]:
%load_ext sql

I connected to Db2 by instantiating an API token which allowed me to run all of my queries through a Jupyter notebook. For the sake of streamlining the project, doing this project through the Jupyter notebooks on Watson Labs allows for all of the IBM technologies to come packaged without having to reinstall each time I run the notebook.

In [2]:
%sql ibm_db_sa://tjk69312:mkjv%402qs5xgnc01k@dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net:50000/BLUDB

'Connected: tjk69312@BLUDB'

In Db2, I have uploaded the .csv values of all three data setsinto the database.

In [3]:
# Verifying the connection was established with a simple query
%sql select count(*) from CHICAGO_CRIME_DATA

 * ibm_db_sa://tjk69312:***@dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net:50000/BLUDB
Done.


1
533


### Sample Queries

In the next few cells, I will demonstrate some of the queries that I wrote in order to fulfill the requests for the assignment.

##### List the top 5 Community Areas by average College Enrollment [number of students]

In [4]:
%sql select COMMUNITY_AREA_NAME, avg(COLLEGE_ENROLLMENT) \
as AVERAGE_COLLEGE_ENROLLMENT \
from CHICAGO_PUBLIC_SCHOOLS \
group by COMMUNITY_AREA_NAME \
order by avg(COLLEGE_ENROLLMENT) desc limit 5

 * ibm_db_sa://tjk69312:***@dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_name,average_college_enrollment
ARCHER HEIGHTS,2411.500000
MONTCLARE,1317.000000
WEST ELSDON,1233.333333
BRIGHTON PARK,1205.875000
BELMONT CRAGIN,1198.833333


##### Use a sub-query to determine which Community Area has the least value for school Safety Score?

In [5]:
%sql select COMMUNITY_AREA_NUMBER, COMMUNITY_AREA_NAME, SAFETY_SCORE \
from CHICAGO_PUBLIC_SCHOOLS \
where SAFETY_SCORE = (select min(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS)

 * ibm_db_sa://tjk69312:***@dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net:50000/BLUDB
Done.


community_area_number,community_area_name,safety_score
40,WASHINGTON PARK,1


##### Which schools in Community Areas 10 to 15 are healthy school certified?

In [6]:
%sql select NAME_OF_SCHOOL, COMMUNITY_AREA_NUMBER from CHICAGO_PUBLIC_SCHOOLS \
where COMMUNITY_AREA_NUMBER between 10 and 15 \
and HEALTHY_SCHOOL_CERTIFIED = 'Yes'

 * ibm_db_sa://tjk69312:***@dashdb-txn-sbox-yp-dal09-12.services.dal.bluemix.net:50000/BLUDB
Done.


name_of_school,community_area_number
Rufus M Hitch Elementary School,10


### Using Pandas Dataframes

In [7]:
# Run matplotlib to graph some of the schools on scatter plot
# Centered around Community_area, 77 different neighborhoods
# From CHICAGO_PUBLIC_SCHOOLS: Safety_Score, Rate_of_Misconduct, College_Enrollment_rate
# From CENSUS_DATA: Percent_of_housing_crowded, Perent_of_households_below_poverty, per_capita_income, hardship_index
# From CHICAGO_CRIME_DATA: Community

I will use the Pandas library to organize the data within the notebook environment. For the visualization of the data, I will be using the Python libraries Matplotlib and Numpy.

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Then I created an localized instance of the data within the notebook for each data set.

In [9]:
CENSUS_DATA_local = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera/data/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012-v2.csv')

In [10]:
CHICAGO_PUBLIC_SCHOOLS_local = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera/data/Chicago_Public_Schools_-_Progress_Report_Cards__2011-2012-v3.csv')

In [11]:
CHICAGO_CRIME_DATA_local = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera/data/Chicago_Crime_Data-v2.csv')

To normalize the column names, I used string methods. Making the columns more uniform allows for less headaches down the road. Also, lowercase columns are easier on my eyes.

In [12]:
CENSUS_DATA_local.columns = CENSUS_DATA_local.columns.str.replace(" ", "_")
CENSUS_DATA_local.columns = CENSUS_DATA_local.columns.str.lower()
CHICAGO_PUBLIC_SCHOOLS_local.columns = CHICAGO_PUBLIC_SCHOOLS_local.columns.str.replace(" ", "_")
CHICAGO_PUBLIC_SCHOOLS_local.columns = CHICAGO_PUBLIC_SCHOOLS_local.columns.str.replace("%", "")
CHICAGO_PUBLIC_SCHOOLS_local.columns = CHICAGO_PUBLIC_SCHOOLS_local.columns.str.replace(",", "")
CHICAGO_PUBLIC_SCHOOLS_local.columns = CHICAGO_PUBLIC_SCHOOLS_local.columns.str.lower()
CHICAGO_CRIME_DATA_local.columns = CHICAGO_CRIME_DATA_local.columns.str.replace(" ", "_")
CHICAGO_CRIME_DATA_local.columns = CHICAGO_CRIME_DATA_local.columns.str.lower()

To simplify the code moving forward, I decided to shorten the names when creating Pandas dataframes.

In [13]:
df1 = pd.DataFrame(data=CENSUS_DATA_local)
df2 = pd.DataFrame(data=CHICAGO_PUBLIC_SCHOOLS_local)
df3 = pd.DataFrame(data=CHICAGO_CRIME_DATA_local)

Now that the dataframes are prepared and more easily accessed through localized instances within Pandas dataframes, listing out the column names here will allow comparisons across separate data frames.

In [14]:
print("Census Data (df1) Columns")
print(df1.columns.tolist())

Census Data (df1) Columns
['community_area_number', 'community_area_name', 'percent_of_housing_crowded', 'percent_households_below_poverty', 'percent_aged_16+_unemployed', 'percent_aged_25+_without_high_school_diploma', 'percent_aged_under_18_or_over_64', 'per_capita_income_', 'hardship_index']


In [15]:
print("Chicago Public Schools (df2) Columns")
print(df2.columns.tolist())

Chicago Public Schools (df2) Columns
['school_id', 'name_of_school', 'elementary_middle_or_high_school', 'street_address', 'city', 'state', 'zip_code', 'phone_number', 'link_', 'network_manager', 'collaborative_name', 'adequate_yearly_progress_made?_', 'track_schedule', 'cps_performance_policy_status', 'cps_performance_policy_level', 'healthy_school_certified', 'safety_icon_', 'safety_score', 'family_involvement_icon', 'family_involvement_score', 'environment_icon_', 'environment_score', 'instruction_icon_', 'instruction_score', 'leaders_icon_', 'leaders_score_', 'teachers_icon_', 'teachers_score', 'parent_engagement_icon_', 'parent_engagement_score', 'parent_environment_icon', 'parent_environment_score', 'average_student_attendance', 'rate_of_misconducts_(per_100_students)_', 'average_teacher_attendance', 'individualized_education_program_compliance_rate_', 'pk-2_literacy_', 'pk-2_math_', 'gr3-5_grade_level_math_', 'gr3-5_grade_level_read__', 'gr3-5_keep_pace_read_', 'gr3-5_keep_pace_

In [16]:
print("Chicago Crime Data (df3) Columns")
print(df3.columns.tolist())

Chicago Crime Data (df3) Columns
['id', 'case_number', 'date', 'block', 'iucr', 'primary_type', 'description', 'location_description', 'arrest', 'domestic', 'beat', 'district', 'ward', 'community_area_number', 'fbicode', 'x_coordinate', 'y_coordinate', 'year', 'updatedon', 'latitude', 'longitude', 'location']


I only wish to look at a subset of the columns from all of the dataframes, so I will used the .copy() method to create abridged dataframes. Since I am being loose with naming conventions, I will make the dataframe titles more descriptive.

In [17]:
census = df1[['community_area_number', 'community_area_name', 'percent_of_housing_crowded', 'percent_households_below_poverty', 'per_capita_income_', 'hardship_index']].copy()
census

,community_area_number,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,per_capita_income_,hardship_index
0,1.0,Rogers Park,7.7,23.6,23939,39.0
1,2.0,West Ridge,7.8,17.2,23040,46.0
2,3.0,Uptown,3.8,24.0,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,37524,17.0
4,5.0,North Center,0.3,7.5,57123,6.0
...,...,...,...,...,...,...
73,74.0,Mount Greenwood,1.0,3.4,34381,16.0
74,75.0,Morgan Park,0.8,13.2,27149,30.0
75,76.0,O'Hare,3.6,15.4,25828,24.0
76,77.0,Edgewater,4.1,18.2,33385,19.0


In [18]:
schools = df2[['community_area_number', 'community_area_name', 'police_district', 'graduation_rate_', 'college_enrollment_rate_', 'college_enrollment', 'safety_score', 'family_involvement_score']].copy()
schools

,community_area_number,community_area_name,police_district,graduation_rate_,college_enrollment_rate_,college_enrollment,safety_score,family_involvement_score
0,7,LINCOLN PARK,18,NDA,NDA,813,99.0,99
1,43,SOUTH SHORE,4,NDA,NDA,521,54.0,66
2,70,ASHBURN,8,NDA,NDA,1324,61.0,NDA
3,61,NEW CITY,9,NDA,NDA,556,56.0,44
4,34,ARMOUR SQUARE,9,NDA,NDA,302,49.0,60
...,...,...,...,...,...,...,...,...
561,61,NEW CITY,9,NDA,NDA,462,32.0,NDA
562,40,WASHINGTON PARK,2,NDA,NDA,371,13.0,49
563,22,LOGAN SQUARE,25,NDA,NDA,748,41.0,NDA
564,42,WOODLAWN,3,NDA,NDA,238,70.0,NDA


# HEAVILY UNDER CONSTRUCTION FROM THIS POINT DOWN

In [19]:
# FIX, FIRST PRIORITY
# Alter dataframe so community_area_number is index and counts are the only column
crime = df3[['id', 'case_number', 'primary_type', 'community_area_number', 'district']].copy()
crime = crime.set_index('community_area_number')
crime

,id,case_number,primary_type,district
community_area_number,,,,
58.0,3512276,HK587712,THEFT,9
23.0,3406613,HK456306,THEFT,11
38.0,8002131,HT233595,THEFT,2
46.0,7903289,HT133522,THEFT,4
66.0,10402076,HZ138551,THEFT,8
...,...,...,...,...
46.0,10453948,HZ192829,HUMAN TRAFFICKING,4
28.0,10397129,HZ133234,NON - CRIMINAL,1
71.0,3269495,HJ747227,RITUALISM,6


In [22]:
combine = [census, schools, crime]
df4 = pd.concat(combine)

df4

,community_area_number,community_area_name,percent_of_housing_crowded,percent_households_below_poverty,per_capita_income_,hardship_index,police_district,graduation_rate_,college_enrollment_rate_,college_enrollment,safety_score,family_involvement_score,id,case_number,primary_type,district
0.0,1.0,Rogers Park,7.7,23.6,23939.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,2.0,West Ridge,7.8,17.2,23040.0,46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,3.0,Uptown,3.8,24.0,35787.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3.0,4.0,Lincoln Square,3.4,10.9,37524.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,5.0,North Center,0.3,7.5,57123.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10453948.0,HZ192829,HUMAN TRAFFICKING,4.0
28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10397129.0,HZ133234,NON - CRIMINAL,1.0
71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3269495.0,HJ747227,RITUALISM,6.0
23.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10840565.0,JA143710,NON-CRIMINAL (SUBJECT SPECIFIED),11.0


In [ ]:
# DELETE

plt.plot(np.arange(10))

### Regression Analysis

In [ ]:
# Use the scatter plots from above and scikit learn to create regression lines to predict the impact of variables on others